In [18]:
import requests     # For retrieving Data from Apidae web site
import json         # For be can getting format json
import pandas as pd
from pandas.io.json import json_normalize   # For transorming 'format json' to 'DataFrame pandas'

project_ID = '4364'     # Cool To Go project id
api_KEY = 'ALrtqQmv'    # Apidae API key
select_id = "86749"     # Example id

def retrieve_data_by_id(project_ID,api_KEY,select_id):
    
    result_df = pd.DataFrame(columns = ['lieu_event','names','telephone','email','site_web','types','longitude','latitude',
                                'adresse1','adresse2','code_postal','ville','description_teaser',
                                'description_','images','publics','categories','accessibilité',
                                'payant','plus_d_infos_et_horaires','date_début','date_fin']) 
    
    url = 'http://api.apidae-tourisme.com/api/v002/objet-touristique/get-by-id/' + select_id + '?'
    url += "responseFields=id,nom,informations,presentation.descriptifCourt,@all"
    url += '&apiKey='+api_KEY
    url += '&projetId='+project_ID
    
    print(url)
    
    re = requests.get(url)
    req = re.json()
    
    dict_for_id = {}

    dict_for_id['lieu_event'] = 'Lieu'
#-----------------------------------------------------------------------------------------------------------------------
    dict_for_id['names'] = None
    if 'gestion' in req:
        if 'membreProprietaire' in req['gestion']:
            if 'nom' in req['gestion']['membreProprietaire']:
                dict_for_id['names'] = req['gestion']['membreProprietaire']['nom']
#-------------------------------------------------------------------------------------------------------------
    dict_for_id['telephone'] = None
    dict_for_id['email'] = None
    dict_for_id['site_web'] = None
    if 'informations' in req:
        if 'moyensCommunication' in req['informations']:
            moyenCommunications = req['informations']['moyensCommunication']
            for values in moyenCommunications :
                if 'type' in values and  'coordonnees' in values :
                    if 'libelleFr' in values['type'] and 'fr' in values['coordonnees'] :
                        if values['type']['libelleFr'] == "Site web (URL)" :
                            dict_for_id['site_web'] = values['coordonnees']['fr']
                        elif values['type']['libelleFr'] == "Téléphone" :
                            dict_for_id['telephone'] = values['coordonnees']['fr']
                        elif values['type']['libelleFr'] == "Mél" :
                            dict_for_id['email'] = values['coordonnees']['fr']
#-----------------------------------------------------------------------------------------------------------------------
    if 'types' in req:
        dict_for_id['types'] = req['type']
    else:
        dict_for_id['types'] = None
#-----------------------------------------------------------------------------------------------------------------------
    dict_for_id['longitude'] = None
    dict_for_id['latitude'] = None
    dict_for_id['adresse2'] = None
    dict_for_id['adresse1'] = None
    if 'localisation' in req:
        if 'geolocalisation' in req['localisation']:
            if 'geoJson' in req['localisation']['geolocalisation']:
                if 'coordinates' in req['localisation']['geolocalisation']['geoJson']:
                    dict_for_id['longitude'] = req['localisation']['geolocalisation']['geoJson']['coordinates'][0]
                    dict_for_id['latitude'] = req['localisation']['geolocalisation']['geoJson']['coordinates'][1]
#------------------------------------------------------------------------------------------------------------------------
        if 'adresse' in req['localisation']:
            if 'adresse1' in req['localisation']['adresse']:
                dict_for_id['adresse1'] = req['localisation']['adresse']['adresse1']
            if 'adresse2' in req['localisation']['adresse']:
                dict_for_id['adresse2'] = req['localisation']['adresse']['adresse2']
#------------------------------------------------------------------------------------------------------------------------
    #dict_for_id['code_postal'] = req['localisation']['adresse']['codePostal']
    if 'localisation' in req:
        if 'adresse' in req['localisation']:
            if 'codePostal' in req['localisation']['adresse']:
                dict_for_id['code_postal'] = req['localisation']['adresse']['codePostal']
            else:
                dict_for_id['code_postal'] = None
        else:
            dict_for_id['code_postal'] = None
    else:
        dict_for_id['code_postal'] = None
#------------------------------------------------------------------------------------------------------------------------
    #dict_for_id['ville'] = req['localisation']['adresse']['commune']['nom']
    if 'localisation' in req:
        if 'adresse' in req['localisation']:
            if 'commune' in req['localisation']['adresse']:
                if 'nom' in req['localisation']['adresse']['commune']:
                    dict_for_id['ville'] = req['localisation']['adresse']['commune']['nom']
                else:
                    dict_for_id['ville'] = None
            else:
                dict_for_id['ville'] = None
        else:
            dict_for_id['ville'] = None
    else:
        dict_for_id['ville'] = None
#------------------------------------------------------------------------------------------------------------------------
    if 'presentation' in req:
        if 'descriptifCourt' in req['presentation']:
            if 'libelleFr' in req['presentation']['descriptifCourt']:
                dict_for_id['description_teaser'] = req['presentation']['descriptifCourt']['libelleFr']
                dict_for_id['description_'] = req['presentation']['descriptifCourt']['libelleFr']
            else:
                dict_for_id['description_teaser'] = None
                dict_for_id['description_'] = None
        else:
            dict_for_id['description_teaser'] = None
            dict_for_id['description_'] = None
    else:
        dict_for_id['description_teaser'] = None
        dict_for_id['description_'] = None
#-----------------------------------------------------------------------------------------------------------------------
    if 'illustrations' in req:
        if 'traductionFichiers' in req['illustrations'][0]:
            if 'url' in req['illustrations'][0]['traductionFichiers'][0]:
                dict_for_id['images'] = req['illustrations'][0]['traductionFichiers'][0]['url']
            else:
                dict_for_id['images'] = None
        else:
            dict_for_id['images'] = None
    else:
        dict_for_id['images'] = None
#-----------------------------------------------------------------------------------------------------------------------
    dict_for_id['publics'] = 'no info'
#-----------------------------------------------------------------------------------------------------------------------
    dict_for_id['categories'] = 'no info'
#-----------------------------------------------------------------------------------------------------------------------
    dict_for_id['accessibilité'] = 'no info'
#-----------------------------------------------------------------------------------------------------------------------
    if 'descriptionTarif' in req:
        if 'gratuit' in req['descriptionTarif']:
            dict_for_id['payant'] = req['descriptionTarif']['gratuit']
        else:
            dict_for_id['payant'] = None
    else:
        dict_for_id['payant'] = None
#-----------------------------------------------------------------------------------------------------------------------
    if 'gestion' in req:
        if 'membreProprietaire' in req['gestion']:
            if 'siteWeb' in req['gestion']['membreProprietaire']:
                dict_for_id['plus_d_infos_et_horaires'] = req['gestion']['membreProprietaire']['siteWeb']
            else:
                dict_for_id['plus_d_infos_et_horaires'] = None
        else:
            dict_for_id['plus_d_infos_et_horaires'] = None
    else:
        dict_for_id['plus_d_infos_et_horaires'] = None
#-----------------------------------------------------------------------------------------------------------------------
    if 'ouverture' in req:
        if 'periodesOuvertures' in req['ouverture']:
            if 'dateDebut' in req['ouverture']['periodesOuvertures'][0]:
                dict_for_id['date_début'] = req['ouverture']['periodesOuvertures'][0]['dateDebut']
            else:
                dict_for_id['date_début'] = None
            if 'dateFin' in req['ouverture']['periodesOuvertures'][0]:
                dict_for_id['date_fin'] = req['ouverture']['periodesOuvertures'][0]['dateFin']
            else:
                dict_for_id['date_fin'] = None
        else:
            dict_for_id['date_début'] = None
            dict_for_id['date_fin'] = None
    else:
        dict_for_id['date_début'] = None
        dict_for_id['date_fin'] = None
  
    #print(dict_for_id)
    result_df = result_df.append(dict_for_id,ignore_index=True)
    
    return result_df

result_df = retrieve_data_by_id(project_ID,api_KEY,select_id)
result_df

http://api.apidae-tourisme.com/api/v002/objet-touristique/get-by-id/86749?responseFields=id,nom,informations,presentation.descriptifCourt,@all&apiKey=ALrtqQmv&projetId=4364


,lieu_event,names,telephone,email,site_web,types,longitude,latitude,adresse1,adresse2,...,description_teaser,description_,images,publics,categories,accessibilité,payant,plus_d_infos_et_horaires,date_début,date_fin
0,Lieu,Vallée de Bozel Tourisme,01 39 33 33 33,jose.hosana48@gmail.com,http://www.ot-bozel.com,None,6.68429,45.442534,Les Voûtes,34 Rue des Tovets,...,"Appartement situé dans un chalet, à 5min de Bo...","Appartement situé dans un chalet, à 5min de Bo...",http://static.apidae-tourisme.com/filestore/ob...,no info,no info,no info,False,http://www.ot-bozel.com,2013-01-01,2013-12-31


In [10]:
count_ = "100"              #"count":20
first = "0"                 # start from 0
selectionId = "86749"
def retrive_data_by_selectionId(project_ID,api_KEY,selectionId):
    import pandas as pd
    result_df = pd.DataFrame(columns = ['lieu_event','names','telephone','email','types','longitude','latitude',
                                'adresse1','adresse2','code_postal','ville','description_teaser',
                                'description_','images','publics','categories','accessibilité',
                                'payant','plus_d_infos_et_horaires','date_début','date_fin']) 
    
    url = 'http://api.apidae-tourisme.com/api/v002/recherche/list-objets-touristiques?query={'
    url += '"projetId":"'+project_ID+'",'
    url += '"apiKey":"'+api_KEY+'",'
    url += '"selectionIds":["'+selectionId+'"],'
    url += '"count":"'+count_+'",'
    url += '"first":"'+str(first)+'"}'

    req = requests.get(url)
    df = pd.json_normalize(req.json(),'objetsTouristiques', errors='ignore')
    for index, row in df.iterrows():
        result_df = result_df.append(retrieve_data_by_id(project_ID,api_KEY,str(row['id'])))
    return result_df

retrive_data_by_selectionId(project_ID,api_KEY,selectionId)

,lieu_event,names,telephone,email,types,longitude,latitude,adresse1,adresse2,code_postal,...,description_teaser,description_,images,publics,categories,accessibilité,payant,plus_d_infos_et_horaires,date_début,date_fin
0,Lieu,Office de Tourisme de Grenoble-Alpes Métropole,NaN,NaN,None,5.802729,45.283682,None,None,38700,...,Bel alpage à 1427m qui est accessible à pied ...,Bel alpage à 1427m qui est accessible à pied ...,http://static.apidae-tourisme.com/filestore/ob...,no info,no info,no info,True,http://www.grenoble-tourisme.com,2016-01-01,2016-12-31
0,Lieu,Office de Tourisme de Grenoble-Alpes Métropole,NaN,NaN,None,5.848310,45.090808,Route de Chamrousse,None,38220,...,"La Réserve Naturelle du lac Luitel, la premièr...","La Réserve Naturelle du lac Luitel, la premièr...",http://static.apidae-tourisme.com/filestore/ob...,no info,no info,no info,True,http://www.grenoble-tourisme.com,2014-01-01,2014-12-31
0,Lieu,Office de Tourisme de Grenoble-Alpes Métropole,NaN,NaN,None,5.763832,45.234888,None,None,38700,...,Accessible en voiture l'été depuis le Sappey-e...,Accessible en voiture l'été depuis le Sappey-e...,http://static.apidae-tourisme.com/filestore/ob...,no info,no info,no info,True,http://www.grenoble-tourisme.com,2016-01-01,2016-12-31
0,Lieu,Office de Tourisme de Grenoble-Alpes Métropole,NaN,NaN,None,5.744369,45.120068,Haute Jarrie,None,38560,...,La Réserve Naturelle Régionale de l'Etang de J...,La Réserve Naturelle Régionale de l'Etang de J...,http://static.apidae-tourisme.com/filestore/ob...,no info,no info,no info,True,http://www.grenoble-tourisme.com,2014-01-01,2014-12-31
0,Lieu,Office de Tourisme de Grenoble-Alpes Métropole,NaN,NaN,None,5.783658,45.084672,Chemin du Plan,None,38220,...,L'étang du Plan de pêche fait partie des balad...,L'étang du Plan de pêche fait partie des balad...,http://static.apidae-tourisme.com/filestore/ob...,no info,no info,no info,True,http://www.grenoble-tourisme.com,2015-01-01,2015-12-31
0,Lieu,Office de Tourisme de Grenoble-Alpes Métropole,NaN,NaN,None,5.845252,45.101070,Route de Chamrousse D111,None,38410,...,Le site de Prémol se situe sur la commune de V...,Le site de Prémol se situe sur la commune de V...,http://static.apidae-tourisme.com/filestore/ob...,no info,no info,no info,True,http://www.grenoble-tourisme.com,2014-01-01,2014-12-31
0,Lieu,Office de Tourisme du Trièves,NaN,NaN,None,5.647963,45.010712,Les saillants,None,38450,...,"Territoire de pêche géré par l'AAPPMA ""La Gaul...","Territoire de pêche géré par l'AAPPMA ""La Gaul...",None,no info,no info,no info,False,http://www.trieves-vercors.fr,2016-03-12,2016-10-09
0,Lieu,Département de l'Isère,NaN,NaN,None,5.796864,45.189851,None,None,38240,...,"Ce site, localisé sur la commune de Meylan, es...","Ce site, localisé sur la commune de Meylan, es...",http://static.apidae-tourisme.com/filestore/ob...,no info,no info,no info,True,http://www.isere.fr,2019-01-01,2019-12-31


In [4]:
import requests     # For retrieving Data from Apidae web site
import json         # For be can getting format json
import pandas as pd
from pandas.io.json import json_normalize   # For transorming 'format json' to 'DataFrame pandas'

project_ID = '4364'     # Cool To Go project id
api_KEY = 'ALrtqQmv'    # Apidae API key
select_id = "86749"     # Example id

def retrieve_data_by_id(project_ID,api_KEY,select_id):
    
    result_df = pd.DataFrame(columns = ['lieu_event','names','types','longitude','latitude',
                                'adresse1','adresse2','code_postal','ville','description_teaser',
                                'description_','images','publics','categories','accessibilité',
                                'payant','plus_d_infos_et_horaires','date_début','date_fin']) 
    
    url = 'http://api.apidae-tourisme.com/api/v002/objet-touristique/get-by-id/' + select_id + '?'
    url += "responseFields=id,nom,informations,presentation.descriptifCourt,@all"
    url += '&apiKey='+api_KEY
    url += '&projetId='+project_ID

    re = requests.get(url)
    req = re.json()
    
print(retrieve_data_by_id(project_ID,api_KEY,select_id))

None
